# Mount the drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!ls drive/'My Drive'/'Colab Notebooks'

 03_CGAN_MNIST.ipynb		    mean-teacher-master
'Chapter_7_SGAN (1).ipynb'	   'mean teacher.v0.mine.ipynb'
'Chapter_7_SGAN (2).ipynb'	   'Mean Teacher.v1.ipynb'
 Chapter_7_SGAN.ipynb		   'SCGAN (1).v1.ipynb'
 Chapter_8_CGAN.ipynb		    SCGAN.3D.test
'Chapter_8_CGAN - 副本 (1).ipynb'   SCGAN.3D.v0.ipynb
'Chapter_8_CGAN - 副本.ipynb'	    SCGAN.v1.ipynb
 dcgan.ipynb			    SSGAN-Tensorflow-master
 full_dataset_vectors.h5	    Untitled0.ipynb
 GAN-tmp.ipynb			    Untitled1.ipynb


In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import json
import time
import h5py
import math

import plotly.graph_objs as go
from matplotlib.pyplot import cm
import plotly.graph_objs as go

import tensorflow as tf
import keras
from keras import backend as K
from keras.layers import (Activation, BatchNormalization, Concatenate, Dense,
                          Dropout, Flatten, Input, Lambda, Reshape, Embedding,
                          Multiply, MaxPool3D)
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose, Conv3D, Conv3DTranspose
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.backend import epsilon

# Process data

In [0]:
with h5py.File('drive/My Drive/Colab Notebooks/full_dataset_vectors.h5', 'r') as mnist3d:
    x_train = mnist3d["X_train"][:]
    x_test = mnist3d["X_test"][:]
    y_train = mnist3d["y_train"][:]
    y_test = mnist3d["y_test"][:]

In [0]:
print ("x_train shape: "+str(x_train.shape))
print ("y_train shape: "+str(y_train.shape))
print ("x_test shape:  "+str(x_test.shape))
print ("y_test shape:  "+str(y_test.shape))

x_train shape: (10000, 4096)
y_train shape: (10000,)
x_test shape:  (2000, 4096)
y_test shape:  (2000,)


In [17]:
# Split the data
x = np.vstack((x_train, x_test))
y = np.hstack((y_train, y_test))
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(9600, 4096)
(9600,)
(2400, 4096)
(2400,)


In [0]:
## Introduce the channel dimension in the input dataset , just shape, no data
xtrain = np.ndarray((x_train.shape[0], 4096, 3))
xtest = np.ndarray((x_test.shape[0], 4096, 3))

## iterate in train and test, add the rgb dimention 
def add_rgb_dimention(array):
    scaler_map = cm.ScalarMappable(cmap="Oranges")
    array = scaler_map.to_rgba(array)[:, : -1]
    return array
for i in range(x_train.shape[0]):
    xtrain[i] = add_rgb_dimention(x_train[i])
for i in range(x_test.shape[0]):
    xtest[i] = add_rgb_dimention(x_test[i])

## convert to 1 + 4D space (1st argument represents number of rows in the dataset)
xtrain = xtrain.reshape(x_train.shape[0], 16, 16, 16, 3)
xtest = xtest.reshape(x_test.shape[0], 16, 16, 16, 3)

## convert target variable into one-hot
ytrain = to_categorical(y_train, num_classes=10)
ytest = to_categorical(y_test, num_classes=10)

In [19]:
print ("xtrain shape: "+str(xtrain.shape))
print ("ytrain shape: "+str(ytrain.shape))
print ("xtest shape:  "+str(xtest.shape))
print ("ytest shape:  "+str(ytest.shape))

xtrain shape: (9600, 16, 16, 16, 3)
ytrain shape: (9600, 10)
xtest shape:  (2400, 16, 16, 16, 3)
ytest shape:  (2400, 10)


In [0]:
class Dataset:
    def __init__(self, num_labeled):
        self.num_labeled = num_labeled
        self.x_train = xtrain
        self.y_train = ytrain
        self.x_test = xtest
        self.y_test = ytest

    def batch_labeled(self, batch_size):
        # Get a random batch of labeled images and their labels
        idx = np.random.randint(0, self.num_labeled, batch_size)
        imgs = self.x_train[idx]
        labels = self.y_train[idx]
        return imgs, labels

    def batch_unlabeled(self, batch_size):
        # Get a random batch of unlabeled images
        idx = np.random.randint(self.num_labeled, self.x_train.shape[0],
                                batch_size)
        imgs = self.x_train[idx]
        return imgs

    def training_set(self):
        # x_train = self.x_train[range(self.num_labeled)]
        # y_train = self.y_train[range(self.num_labeled)]
        return self.x_train, self.y_train

    def test_set(self):
        return self.x_test, self.y_test

# Build the model

In [0]:
img_rows = 16
img_cols = 16
img_deps = 16
channels = 3

# Input image dimensions
img_shape = (img_rows, img_cols, img_deps, channels)

# Size of the noise vector, used as input to the Generator
z_dim = 100

# Number of classes in the dataset
num_classes = 10

In [0]:
def build_generator(z_dim):
  
    z = Input(shape=(z_dim, ))
    label = Input(shape=(num_classes, ), dtype='float32')
    label_embedding = Dense(z_dim, input_dim=num_classes)(label)
    joined_representation = Multiply()([z, label_embedding])
    
#     model = Sequential()

    # Reshape input into 4x4x4x256 tensor via a fully connected layer
    model = Dense(256 * 4 * 4 * 4, input_dim=z_dim)(joined_representation)
    model = Reshape((4, 4, 4, 256))(model)

    # Transposed convolution layer, from 4x4x4x256 into 8x8x8x128 tensor
    model = Conv3DTranspose(128, kernel_size=3, strides=2, padding='same')(model)

    # Batch normalization
    model = BatchNormalization()(model)

    # Leaky ReLU activation
    model = LeakyReLU(alpha=0.01)(model)

    # Transposed convolution layer, from 8x8x8x128 to 8x8x8x64 tensor
    model = Conv3DTranspose(64, kernel_size=3, strides=1, padding='same')(model)

    # Batch normalization
    model = BatchNormalization()(model)

    # Leaky ReLU activation
    model = LeakyReLU(alpha=0.01)(model)

    # Transposed convolution layer, from 8x8x8x64 to 16x16x16x3 tensor
    model = Conv3DTranspose(3, kernel_size=3, strides=2, padding='same')(model)

    # Output layer with tanh activation
    conditioned_img = Activation('tanh')(model)
    
#     conditioned_img = model(joined_representation)

    model = Model([z, label], conditioned_img)

    return model

In [51]:
build_generator(z_dim).summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
dense_18 (Dense)                (None, 100)          1100        input_10[0][0]                   
__________________________________________________________________________________________________
multiply_3 (Multiply)           (None, 100)          0           input_9[0][0]                    
                                                                 dense_18[0][0]             

In [0]:
def build_discriminator_net(img_shape):

    model = Sequential()

    # Convolutional layer, from 16x16x16x3 into 8x8x8x8 tensor
    model.add(Conv3D(8, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.01))

    # Convolutional layer, from 8x8x8x8 into 4x4x4x16 tensor
    model.add(Conv3D(32, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.5))

    # Convolutional layer, from 4x4x4x16 tensor into 2x2x2x32 tensor
    model.add(Conv3D(32, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.5))

    # Convolutional layer, from 2x2x2x32 tensor into 1x1x1x64 tensor
    model.add(Conv3D(64, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(num_classes))

    return model

In [0]:
def build_discriminator_net(img_shape):
    
    model = Sequential()

    model.add(Conv3D(filters=8, kernel_size=(3, 3, 3), use_bias=False, , input_shape=img_shapepadding='Same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv3D(filters=16, kernel_size=(3, 3, 3), use_bias=False, padding='Same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    model.add(MaxPool3D(pool_size=(2, 2, 2))) # the pool_size (2, 2, 2) halves the size of its input

    model.add(Conv3D(filters=32, kernel_size=(3, 3, 3), use_bias=False, padding='Same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv3D(filters=64, kernel_size=(3, 3, 3), use_bias=False, padding='Same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    ## Pooling layer
    model.add(MaxPool3D(pool_size=(2, 2, 2)))
    model.add(Dropout(0.25)) #No more BatchNorm after this layer because we introduce Dropout

    model.add(Flatten())

    ## Dense layers
    model.add(Dense(units=4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=10, activation='softmax'))

    return model

In [106]:
build_discriminator_net(img_shape).summary()

Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_70 (Conv3D)           (None, 8, 8, 8, 8)        656       
_________________________________________________________________
leaky_re_lu_82 (LeakyReLU)   (None, 8, 8, 8, 8)        0         
_________________________________________________________________
conv3d_71 (Conv3D)           (None, 4, 4, 4, 32)       6944      
_________________________________________________________________
batch_normalization_60 (Batc (None, 4, 4, 4, 32)       128       
_________________________________________________________________
leaky_re_lu_83 (LeakyReLU)   (None, 4, 4, 4, 32)       0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 4, 4, 4, 32)       0         
_________________________________________________________________
conv3d_72 (Conv3D)           (None, 2, 2, 2, 32)     

In [0]:
def build_discriminator_class(discriminator_net):

    model = Sequential()

    model.add(discriminator_net)

    # Softmax activation, giving predicted probability distribution over the real classes
    model.add(Activation('softmax'))

    return model

In [0]:
def build_discriminator_realOrFake(discriminator_net):

    model = Sequential()

    model.add(discriminator_net)

    def predict(x):
        # Transform distribution over real classes into a binary real-vs-fake probability
        prediction = 1.0 - (1.0 / (K.sum(K.exp(x), axis=-1, keepdims=True) + 1.0))
        return prediction

    # 'Real-vs-fake' output neuron defined above
    model.add(Lambda(predict))

    return model

In [0]:
def build_gan(generator, discriminator):
    
    z = Input(shape=(z_dim, ))
    label = Input(shape=(num_classes, ))
    img = generator([z, label])
    output = discriminator(img)
    model = Model([z, label], output)

    return model

In [0]:
# Core Discriminator network:
# These layers are shared during supervised and unsupervised training

discriminator_net = build_discriminator_net(img_shape)


discriminator_realOrFake = build_discriminator_realOrFake(discriminator_net)
discriminator_realOrFake.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam())

discriminator_class = build_discriminator_class(discriminator_net)
discriminator_class.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam())

In [0]:
# Build the Generator
generator = build_generator(z_dim)

discriminator_realOrFake.trainable = False
discriminator_class.trainable = False

gan_realOrFake = build_gan(generator, discriminator_realOrFake)
# gan_realOrFake.compile(loss=loss_d_realOrFake, metrics=['accuracy'], optimizer=Adam())
gan_realOrFake.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam())

gan_class = build_gan(generator, discriminator_class)
# gan_class.compile(loss=loss_d_class, metrics=['accuracy'], optimizer=Adam())
gan_class.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam())

In [89]:
discriminator_realOrFake.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_39 (Sequential)   (None, 10)                91802     
_________________________________________________________________
lambda_15 (Lambda)           (None, 1)                 0         
Total params: 183,348
Trainable params: 91,546
Non-trainable params: 91,802
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning:

Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?



In [90]:
discriminator_class.summary()

Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_39 (Sequential)   (None, 10)                91802     
_________________________________________________________________
activation_11 (Activation)   (None, 10)                0         
Total params: 183,348
Trainable params: 91,546
Non-trainable params: 91,802
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning:

Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?



In [91]:
gan_realOrFake.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
model_12 (Model)                (None, 16, 16, 16, 3 2767951     input_25[0][0]                   
                                                                 input_26[0][0]                   
__________________________________________________________________________________________________
sequential_40 (Sequential)      (None, 1)            91802       model_12[1][0]            

In [92]:
gan_class.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_28 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
model_12 (Model)                (None, 16, 16, 16, 3 2767951     input_27[0][0]                   
                                                                 input_28[0][0]                   
__________________________________________________________________________________________________
sequential_41 (Sequential)      (None, 10)           91802       model_12[2][0]            

In [93]:
generator.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
dense_29 (Dense)                (None, 100)          1100        input_24[0][0]                   
__________________________________________________________________________________________________
multiply_6 (Multiply)           (None, 100)          0           input_23[0][0]                   
                                                                 dense_29[0][0]            

# Training

In [64]:
%mkdir models
%mkdir losses
%mkdir models/models-label-1000
%mkdir losses/losses-label-1000

mkdir: cannot create directory ‘models’: File exists
mkdir: cannot create directory ‘losses’: File exists
mkdir: cannot create directory ‘models/models-label-1000’: File exists
mkdir: cannot create directory ‘losses/losses-label-1000’: File exists


In [0]:
def pretrain(iterations_pre, batch_size, save_interval):
  for iteration in range(iterations_pre):
      imgs_labeled, labels = dataset.training_set()
      # imgs_labeled, labels = dataset.batch_labeled(1000)
      loss_d_labeled_class, acc_d_labeled_class = discriminator_class.train_on_batch(imgs_labeled, labels)
      if (iteration + 1) % save_interval == 0:
          
          # Output training progress
          print(
              "%d [D loss class: %.4f, acc: %.2f%%]"
              % (iteration + 1, loss_d_labeled_class, 100 * acc_d_labeled_class))
          
          discriminator_class.save("./models/discriminator_class.h5")
          x, y = dataset.training_set()
          _, accuracy = discriminator_class.evaluate(x, y)
          print("Training Accuracy: %.2f%%" % (100 * accuracy))      

In [120]:
# Set hyperparameters
iterations_pre = 2000
batch_size = 32
save_interval = 100

dataset = Dataset(num_labeled)

pretrain(iterations_pre, batch_size, save_interval)

100 [D loss class: 1.1383, acc: 61.46%]
9600/9600 [==============================] - 1s 136us/step
Training Accuracy: 70.32%
200 [D loss class: 1.1363, acc: 61.18%]
9600/9600 [==============================] - 1s 136us/step
Training Accuracy: 71.03%
300 [D loss class: 1.1165, acc: 62.04%]
9600/9600 [==============================] - 1s 137us/step
Training Accuracy: 71.41%
400 [D loss class: 1.1007, acc: 62.36%]
9600/9600 [==============================] - 1s 136us/step
Training Accuracy: 71.70%
500 [D loss class: 1.0922, acc: 62.77%]
9600/9600 [==============================] - 1s 140us/step
Training Accuracy: 72.58%
600 [D loss class: 1.0695, acc: 63.00%]
9600/9600 [==============================] - 1s 142us/step
Training Accuracy: 73.09%
700 [D loss class: 1.0833, acc: 63.10%]
9600/9600 [==============================] - 1s 144us/step
Training Accuracy: 73.44%
800 [D loss class: 1.0577, acc: 64.16%]
9600/9600 [==============================] - 1s 133us/step
Training Accuracy: 74.49%


In [122]:
x, y = dataset.test_set()
_, accuracy = discriminator_class.evaluate(x, y)
print("Test Accuracy: %.2f%%" % (100 * accuracy))

2400/2400 [==============================] - 0s 176us/step
Test Accuracy: 65.29%


In [0]:
def train(iterations, batch_size, save_interval, m, n):


    # Labels for real images: all ones
    real = np.ones((batch_size, 1))

    # Labels for fake images: all zeros
    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):

        for _ in range(m):

            # -------------------------
            #  Train the Discriminator
            # -------------------------


            # Get labeled examples
            imgs_labeled, labels = dataset.batch_labeled(batch_size)

            # Get unlabeled examples
            imgs_unlabeled = dataset.batch_unlabeled(batch_size)

            # Generate a batch of fake images
            z = np.random.normal(0, 1, (batch_size, z_dim))
            fake_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
            fake_labels = to_categorical(fake_labels, num_classes=num_classes)
            imgs_gen = generator.predict([z, fake_labels])

            # Train discriminator
            loss_d_labeled_realOrFake, acc_d_labeled_realOrFake = discriminator_realOrFake.train_on_batch(imgs_labeled, real)
            loss_d_unlabeled_realOrFake, acc_d_unlabeled_realOrFake = discriminator_realOrFake.train_on_batch(imgs_unlabeled, real)
            loss_d_gen_realOrFake, acc_d_gen_realOrFake = discriminator_realOrFake.train_on_batch(imgs_gen, fake)
            loss_d_labeled_class, acc_d_labeled_class = discriminator_class.train_on_batch(imgs_labeled, labels)
            loss_d_gen_class, acc_d_gen_class = discriminator_class.train_on_batch(imgs_gen, fake_labels)

            # Calculate loss and acc
            loss_d_train_realOrFake = np.add(loss_d_labeled_realOrFake, loss_d_unlabeled_realOrFake)
            loss_d_class = np.add(loss_d_labeled_class, loss_d_gen_class)
            loss_d_class = loss_d_labeled_class
            loss_d_realOrFake = np.add(loss_d_gen_realOrFake, np.add(loss_d_labeled_realOrFake, loss_d_unlabeled_realOrFake))
            loss_d = np.add(loss_d_class, loss_d_realOrFake)
            acc_d_class = np.add(acc_d_labeled_class, acc_d_gen_class)/2.0
            acc_d_class = acc_d_labeled_class
            acc_d_realOrFake = np.add(acc_d_gen_realOrFake, np.add(acc_d_labeled_realOrFake, acc_d_unlabeled_realOrFake))/3.0
            acc_d = np.add(acc_d_class, acc_d_realOrFake)/2.0

        for _ in range(n):

            # ---------------------
            #  Train the Generator
            # ---------------------

            # Generate a batch of fake images
            z = np.random.normal(0, 1, (batch_size, z_dim))
            fake_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
            fake_labels = to_categorical(fake_labels, num_classes=num_classes)

            # Train Generator
            loss_g_realOrFake, acc_g_realOrFake = gan_realOrFake.train_on_batch([z, fake_labels], fake)
            # loss_g_class, acc_g_class = gan_class.train_on_batch([z, fake_labels], fake_labels)

            # Calculate loss
            # loss_g = np.add(loss_g_realOrFake, loss_g_class)
            loss_g = losses_g_realOrFake

        if (iteration + 1) % save_interval == 0:

            # Save Discriminator supervised classification loss to be plotted after training
            losses_d_class.append(loss_d_class)
            losses_d_realOrFake.append(loss_d_realOrFake)
            # losses_g_class.append(loss_g_class)
            losses_g_realOrFake.append(loss_g_realOrFake)
            
            iteration_checkpoints.append(iteration + 1)

            # Output training progress
            print(
                "%d [D loss class: %.4f, acc: %.2f%%] [D loss realOrFake: %.4f, acc:%.2f%%] [G loss realOrFake: %.4f, acc: %.2f%%]"
                % (iteration + 1, 
                    loss_d_class, 100 * acc_d_class,
                    loss_d_realOrFake, 100 * acc_d_realOrFake, 
                    # loss_g_class, 100 * acc_g_class,
                    loss_g_realOrFake, 100 * acc_g_realOrFake))
            
            discriminator_class.save("./models/models-label-" + str(num_labeled) + "/discriminator_class-" + str(iteration+1) + ".h5")
            discriminator_realOrFake.save("./models/models-label-" + str(num_labeled) + "/discriminator_realOrFake-" + str(iteration+1) + ".h5")
            generator.save("./models/models-label-" + str(num_labeled) + "/generator-" + str(iteration+1) + ".h5")
            file1 = "./losses/losses-label-" + str(num_labeled) + "/losses_d_class.json"
            file2 = "./losses/losses-label-" + str(num_labeled) + "/losses_d_realOrFake.json"
            # file3 = "./losses/losses-label-" + str(num_labeled) + "/losses_g_class.json"
            file4 = "./losses/losses-label-" + str(num_labeled) + "/losses_g_realOrFake.json"
            with open(file1, 'w') as json_file:
                  json.dump(str(losses_d_class), json_file)
            with open(file2, 'w') as json_file:
                  json.dump(str(losses_d_realOrFake), json_file)
            # with open(file3, 'w') as json_file:
            #       json.dump(str(losses_g_class), json_file)
            with open(file4, 'w') as json_file:
                  json.dump(str(losses_g_realOrFake), json_file)
            
            x,y = dataset.training_set()
            _, acc = discriminator_class.evaluate(x,y)
            print(str(100*acc)+"%")

In [126]:
# Set hyperparameters
iterations = 8000 # 8000
batch_size = 32
save_interval = 100
num_labeled = 1000  # Number of labeled examples to use (rest will be used as unlabeled)
m = 1 # iteration of Discriminator
n = 1 # iteration of Generator

# dataset = Dataset(num_labeled)

# losses_d_labeled_realOrFake = []
# losses_d_unlabeled_realOrFake = []
# losses_d_gen_realOrFake = []
# losses_d_labeled_class = []
# losses_d_gen_class = []
# losses_d_train_realOrFake = []
losses_d_class = []
losses_d_realOrFake = []
losses_d = []
losses_g_realOrFake = []
losses_g_class = []
losses_g = []

iteration_checkpoints = []

starttime = time.clock()

# Train the SGAN for the specified number of iterations
train(iterations, batch_size, save_interval, m, n)

endtime = time.clock()
print("Training time: %.4fs" % (endtime - starttime))

100 [D loss class: 1.0212, acc: 84.38%] [D loss realOrFake: 1.0365, acc:87.50%] [G loss realOrFake: 2.0127, acc: 9.38%]
9600/9600 [==============================] - 1s 144us/step
48.354166666666664%
200 [D loss class: 1.3116, acc: 53.12%] [D loss realOrFake: 0.7176, acc:95.83%] [G loss realOrFake: 1.6556, acc: 3.12%]
9600/9600 [==============================] - 1s 139us/step
57.69791666666667%
300 [D loss class: 1.3318, acc: 56.25%] [D loss realOrFake: 0.5190, acc:98.96%] [G loss realOrFake: 1.7475, acc: 0.00%]
9600/9600 [==============================] - 1s 140us/step
26.447916666666664%
400 [D loss class: 0.9919, acc: 68.75%] [D loss realOrFake: 0.3736, acc:98.96%] [G loss realOrFake: 1.3394, acc: 0.00%]
9600/9600 [==============================] - 1s 136us/step
49.604166666666664%
500 [D loss class: 1.0817, acc: 62.50%] [D loss realOrFake: 0.3263, acc:98.96%] [G loss realOrFake: 1.3385, acc: 59.38%]
9600/9600 [==============================] - 1s 141us/step
52.416666666666664%
600 [

# Evaluate

In [127]:
x, y = dataset.training_set()

tmodel = keras.models.load_model("./models/models-label-" + str(num_labeled) + "/discriminator_class-5700.h5")

# Compute classification accuracy on the training set
_, accuracy = tmodel.evaluate(x, y)
# _, accuracy = discriminator_class.evaluate(x, y)
print("Training Accuracy: %.2f%%" % (100 * accuracy))

9600/9600 [==============================] - 3s 322us/step
Training Accuracy: 53.15%


In [128]:
x, y = dataset.test_set()

# tmodel = keras.models.load_model("./models/models-label-" + str(num_labeled) + "/discriminator_class-5700.h5")

# Compute classification accuracy on the training set
_, accuracy = tmodel.evaluate(x, y)
# _, accuracy = discriminator_class.evaluate(x, y)
print("Training Accuracy: %.2f%%" % (100 * accuracy))

2400/2400 [==============================] - 0s 139us/step
Training Accuracy: 48.42%
